In [65]:
import talib as ta
import numpy as np
import matplotlib.pyplot as plt
import finpy_tse as fpy
import hvplot.pandas  # noqa
import holoviews as hv
import pandas as pd
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, show

In [24]:
class TA:

    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date

    def get_price(self):
        price_hist = fpy.Get_Price_History(
            stock=self.ticker,
            start_date=self.start_date,
            end_date=self.end_date,
            ignore_date=False,
            adjust_price=True,
            show_weekday=False,
            double_date=True).rename(columns=str.lower)
        price_hist = price_hist[["date", "adj open","adj high","adj low","adj close","adj final","volume", "value",]]
        price_hist.iloc[:,1:6] = price_hist.iloc[:,1:6].div(10).astype(int)
        price_hist.columns = price_hist.columns.str.replace("adj ","", regex=True)
        price_hist["date_"] = pd.date_range(end=price_hist["date"][-1], periods=len(price_hist), )
        return price_hist

In [56]:
ticker = TA(ticker="قصفها", start_date="1395-01-30", end_date="1401-09-30")

In [58]:
df = ticker.get_price()
df.set_index("date", inplace=True)

In [59]:
ticker.ticker

'قصفها'

In [60]:
short_ema = 21
long_ema = 52
df[f"EMA_{short_ema}"] = ta.EMA(df["final"], short_ema)
df[f"EMA_{long_ema}"] = ta.EMA(df["final"], long_ema)
df["RSI"] = ta.RSI(df["final"])
df["ln"] = ta.LN(df["final"])

In [68]:

ohlc = df.hvplot.ohlc('date_', ['open', 'high','low', 'close'], ylabel='Price (IRT)', grid=True, xaxis=None, yticks=df["final"], logy=True, width=1200 , label = ticker.ticker)
upper_limit = hv.HLine(75).opts(
    color='red',
    line_dash='dashed',
    line_width=0.75,
)
lower_limit = hv.HLine(25).opts(
    color='green',
    line_dash='dashed',
    line_width=0.75,
)
median_limit = hv.HLine(50).opts(
    color='silver',
    line_dash='dashed',
    line_width=0.,
)
short_ema_ = df.hvplot(x="date_",y=f"EMA_{short_ema}",label=f"EMA_{short_ema}", alpha=0.5)
long_ema_ = df.hvplot(x="date_",y=f"EMA_{long_ema}", label=f"EMA_{long_ema}", alpha=0.5)
rsi = df.hvplot(x="date_",y="RSI", height=150, width=1200) * upper_limit*lower_limit*median_limit
volume = df.hvplot.step(x='date_', y='volume', height=80, xaxis=None)

layout = ( ohlc*short_ema_*long_ema_ + rsi).cols(1)
layout.opts(merge_tools=True)

:Layout
   .Overlay.I  :Overlay
      .Segments.I   :Segments   [date_,low,date_,high]   (open,high,low,close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,open,ubound,close]   (open,high,low,close,lbound,ubound)
      .Curve.EMA_21 :Curve   [date_]   (EMA_21)
      .Curve.EMA_52 :Curve   [date_]   (EMA_52)
   .Overlay.II :Overlay
      .Curve.I   :Curve   [date_]   (RSI)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]
      .HLine.III :HLine   [x,y]

In [131]:
ticker.reset_index(inplace=True)

In [132]:
p = figure(width=1080, height=600, x_axis_type="datetime")
p.title.text = 'Click on legend entries to hide the corresponding lines'
p.line(ticker.index, ticker['ln'], line_width=2,  alpha=0.8, legend_label="name")

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
dict(ticker["final"])

In [1]:
import os
from datetime import datetime
import pandas_datareader.data as web

In [20]:
import requests
proxies = {
    'http': 'http://209.182.239.62:80'
}
session = requests.Session()
session.proxies.update(proxies)

In [24]:
import pandas_datareader.data as web

import datetime

start = datetime.datetime(2010, 1, 1)

end = datetime.datetime(2022, 11, 27)

gdp = web.DataReader('GDP','fred', start, end)

In [34]:
df = web.DataReader('BTC-USD', 'yahoo', start='2019-09-10', end='2019-10-09', interal="15m")

TypeError: DataReader() got an unexpected keyword argument 'interal'

In [35]:
import yfinance as yf

In [110]:
df = yf.download(tickers='BTC-USD', period = '1000D', interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [111]:
df = df.iloc[:,:4]
df.reset_index(inplace=True)

In [112]:
df

,Date,Open,High,Low,Close
0,2020-03-03,8865.387695,8901.598633,8704.990234,8787.786133
1,2020-03-04,8788.541992,8843.366211,8712.431641,8755.246094
2,2020-03-05,8760.285156,9142.054688,8757.253906,9078.762695
3,2020-03-06,9078.308594,9167.695312,9032.079102,9122.545898
4,2020-03-07,9121.600586,9163.220703,8890.744141,8909.954102
...,...,...,...,...,...
994,2022-11-22,15782.300781,16253.047852,15656.606445,16189.769531
995,2022-11-23,16195.588867,16638.193359,16170.502930,16610.707031
996,2022-11-24,16611.636719,16771.474609,16501.767578,16604.464844
997,2022-11-25,16602.269531,16603.316406,16388.404297,16521.841797


In [123]:
#df["date"] = pd.to_datetime(df.index)

inc = df.Close > df.Open
dec = df.Open > df.Close

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(tools=TOOLS, width=1300, height=700,
           title="MSFT Candlestick without missing dates",
           background_fill_color="#efefef")
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05

# map dataframe indices to date strings and use as label overrides
p.xaxis.major_label_overrides = {
    i: date.strftime('%b %d') for i, date in zip(df.index,df["Date"])
}
#
# # one tick per week (5 weekdays)
p.xaxis.ticker = list(range(df.index[0], df.index[-1], 4))

p.segment(df.index, df.High, df.index, df.Low, color="black")

p.vbar(df.index[dec], 0.6, df.Open[dec], df.Close[dec], color="#eb3c40" )
p.vbar(df.index[inc], 0.6, df.Open[inc], df.Close[inc], fill_color="white",
       line_color="#49a3a3", line_width=2)
show(p)

AttributeError: unexpected attribute 'logy' to VBar, possible attributes are bottom, fill_alpha, fill_color, hatch_alpha, hatch_color, hatch_extra, hatch_pattern, hatch_scale, hatch_weight, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, syncable, tags, top, width or x

In [124]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show


df = pd.DataFrame({"date":['2017-01-01','2017-01-02','2017-01-03','2017-01-06','2017-01-07']})
df["value"] = [10,20,15,30,25]
print(df)
width = 24*60*60*100
TOOLS = 'hover,crosshair,pan,wheel_zoom,box_zoom,reset,save,box_select'
picture = figure(width=1000, height=400, tools=TOOLS, x_axis_type='datetime')
picture.vbar(pd.to_datetime(df.date), width,df['value'],0, color='blue', alpha=0.5)
show(picture)

         date  value
0  2017-01-01     10
1  2017-01-02     20
2  2017-01-03     15
3  2017-01-06     30
4  2017-01-07     25
